In [12]:
import os
os.environ['HF_ENDPOINT']='https://hf-mirror.com'
os.environ['PYTORCH_CUDA_ALLOC_CONF']='expandable_segments:True'
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO']='0.0'
import argparse
import json
import time
import torch

from collections import defaultdict

from surya.input.langs import replace_lang_with_code
from surya.input.load import load_from_folder, load_from_file, load_lang_file
from surya.model.detection.model import load_model as load_detection_model, load_processor as load_detection_processor
from surya.model.recognition.model import load_model as load_recognition_model
from surya.model.recognition.processor import load_processor as load_recognition_processor
from surya.ocr import run_ocr
from surya.postprocessing.text import draw_text_on_image
from surya.settings import settings

import sys

import numpy as np
import pandas as pd
class box_text:
    left_top_x=0
    left_top_y=0
    right_bottom_x=0
    right_bottom_y =0
    text=""
    x=0
    y=0
    def __init__(self,left_top_x,left_top_y,right_bottom_x,right_bottom_y,text):
        self.left_top_x=left_top_x
        self.left_top_y=left_top_y
        self.right_bottom_x=right_bottom_x
        self.right_bottom_y =right_bottom_y
        self.text=text
    def set_coordinate(self,x,y):
        self.x = x
        self.y=y
def generate_csv(bboxes,texts,file_path):
    length = len(bboxes)
    last_top = None
    last_bottom = None
    lines = []
    line = []
    box_text_arr = []
    for idx in range(length):
        bbox = bboxes[idx]
        left_top_x = bbox[0]
        left_top_y = bbox[1]
        right_bottom_x = bbox[2] 
        right_bottom_y = bbox[3]
        text = texts[idx]
        box = box_text(left_top_x,left_top_y,right_bottom_x,right_bottom_y,text)
        box_text_arr.append(box)
    box_text_arr.sort(key=lambda x:x.left_top_y)
    for idx in range(length):
        bbox = box_text_arr[idx]
        left_top_y = bbox.left_top_y
        right_bottom_y = bbox.right_bottom_y
        same_line = False
        if last_bottom is None or ((last_bottom >= left_top_y>=last_top)):
            same_line = True
        if not same_line:
            line.sort(key=lambda x:x.left_top_x)
            lines.append(line)
            line = [bbox]
        else:
            line.append(bbox)
        last_bottom = right_bottom_y
        last_top = left_top_y
    lines.append(line)
    for y in range(len(lines)):
        line = lines[y]
        for x in range(len(line)):
            line[x].set_coordinate(x,y)
    new_lines = []
    for line in lines:
        new_lines.extend(line)


    new_lines.sort(key=lambda x:x.left_top_x)
    last_left=None
    last_right = None
    col_lines = []
    line = []
    for idx in range(len(new_lines)):
        bbox = new_lines[idx]
        same_line = False
        if last_right is None or ((last_right >= bbox.left_top_x>=last_left)):
            same_line = True
        if not same_line:
            line.sort(key=lambda x:x.left_top_y)
            col_lines.append(line)
            line = [bbox]
        else:
            line.append(bbox)
        last_right = bbox.right_bottom_x
        last_left = bbox.left_top_x
    col_lines.append(line)
    col_num = 0
    row_num=0
    for line in col_lines:
        x_cnt = {}
        for item in line:
            if item.x not in x_cnt:
                x_cnt[item.x] = 1
            else:
                x_cnt[item.x] += 1

        xlist = list(x_cnt.keys())
        xlist.sort(key=lambda x:x_cnt[x],reverse=True)
        final_x = xlist[0]
        print(final_x)
        for item in line:

            if '1053138643068149760' in item.text:
                pass
            item.x=final_x
            if item.y>row_num:
                row_num=item.y
            if item.x>col_num:
                col_num=item.x
    final_lines = [[box_text(0,0,0,0,"检测失败") for j in range(col_num+1)] for i in range(row_num+1)]
    for line in col_lines:
        for item in line:
            try:
                final_lines[item.y][item.x] = item
            except Exception as ex:
                raise ValueError(item.x,item.y,ex)
    data={}
    for line in final_lines:
        for idx,item in enumerate(line):
            if idx not in data:
                data[idx]=[] 
            data[idx].append(item.text)
            print(len(line))
    df = pd.DataFrame(data)
    df.to_csv(file_path,index=False)
def save_bboxes_csv(bboxes,pred_texts,target_file):
    length = len(bboxes)
    x_1 = []
    y_1 = []
    x_2 = []
    y_2 = []
    for i in range(length):
        box = bboxes[i]
        x_1.append(box[0])
        y_1.append(box[1])
        x_2.append(box[2])
        y_2.append(box[3])
    df = pd.DataFrame({
        "x_1":x_1,
        "y_1":y_1,
        "x_2":x_2,
        "y_2":y_2,
        "text":pred_texts,
    })
    df.to_csv(target_file,index=False)

In [4]:
path = "/Volumes/dataset1/data/traderecord/eng/processed"
path="/Users/androidjk/projects/surya/results/processed/截屏2024-12-28 00_43_01_7_origin.png"
sys.argv = ['', path, '--results_dir', './results','--image','--debug']
parser = argparse.ArgumentParser(description="Detect bboxes in an input file or folder (PDFs or image).")
parser.add_argument("input_path", type=str, help="Path to pdf or image file or folder to detect bboxes in.")
parser.add_argument("--results_dir", type=str, help="Path to JSON file with OCR results.", default=os.path.join(settings.RESULT_DIR, "surya"))
parser.add_argument("--max", type=int, help="Maximum number of pages to process.", default=None)
parser.add_argument("--start_page", type=int, help="Page to start processing at.", default=0)
parser.add_argument("--images", action="store_true", help="Save images of detected bboxes.", default=False)
parser.add_argument("--langs", type=str, help="Optional language(s) to use for OCR. Comma separate for multiple. Can be a capitalized language name, or a 2-letter ISO 639 code.", default=None)
parser.add_argument("--lang_file", type=str, help="Optional path to file with languages to use for OCR. Should be a JSON dict with file names as keys, and the value being a list of language codes/names.", default=None)
parser.add_argument("--debug", action="store_true", help="Enable debug logging.", default=False)
args = parser.parse_args()

if os.path.isdir(args.input_path):
    images, names, _ = load_from_folder(args.input_path, args.max, args.start_page)
    highres_images, _, _ = load_from_folder(args.input_path, args.max, args.start_page, settings.IMAGE_DPI_HIGHRES)
    folder_name = os.path.basename(args.input_path)
else:
    images, names, _ = load_from_file(args.input_path, args.max, args.start_page)
    highres_images, _, _ = load_from_file(args.input_path, args.max, args.start_page, settings.IMAGE_DPI_HIGHRES)
    folder_name = os.path.basename(args.input_path).split(".")[0]

if args.lang_file:
    # We got all of our language settings from a file
    langs = load_lang_file(args.lang_file, names)
    for lang in langs:
        replace_lang_with_code(lang)
    image_langs = langs
elif args.langs:
    # We got our language settings from the input
    langs = args.langs.split(",")
    replace_lang_with_code(langs)
    image_langs = [langs] * len(images)
else:
    image_langs = [None] * len(images)



In [5]:

det_processor = load_detection_processor()
det_model = load_detection_model()

rec_model = load_recognition_model()
rec_processor = load_recognition_processor()

Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device mps with dtype torch.float16


In [6]:


det_processor = load_detection_processor()
det_model = load_detection_model()

rec_model = load_recognition_model()
rec_processor = load_recognition_processor()


result_path = os.path.join(args.results_dir, folder_name)
os.makedirs(result_path, exist_ok=True)

start = time.time()
torch.cuda.empty_cache()
predictions_by_image = run_ocr(images, image_langs, det_model, det_processor, rec_model, rec_processor, highres_images=highres_images)
if args.debug:
    print(f"OCR took {time.time() - start:.2f} seconds")
    max_chars = max([len(l.text) for p in predictions_by_image for l in p.text_lines])
    print(f"Max chars: {max_chars}")


for idx, (name, image, pred, langs) in enumerate(zip(names, images, predictions_by_image, image_langs)):
    
    bboxes = [l.bbox for l in pred.text_lines]
    pred_text = [l.text for l in pred.text_lines]
    # page_image = draw_text_on_image(bboxes, pred_text, image.size, langs, has_math="_math" in langs if langs else False)
    # page_image.save(os.path.join(result_path, f"{name}_{idx}.png"))
    # image.save(os.path.join(result_path, f"{name}_{idx}_origin.png"))
    # generate_csv(bboxes,pred_text,os.path.join(result_path, f"{name}_{idx}.csv"))

Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device mps with dtype torch.float16


Recognizing Text: 100%|██████████| 12/12 [11:42<00:00, 58.51s/it]

OCR took 703.70 seconds
Max chars: 23


In [13]:
save_bboxes_csv(bboxes,pred_text,"/Users/androidjk/projects/surya/results/processed/截屏2024-12-28 00_43_01_7_origin_box.csv")

In [ ]:

for idx, (name, image, pred, langs) in enumerate(zip(names, images, predictions_by_image, image_langs)):
    bboxes = [l.bbox for l in pred.text_lines]
    pred_text = [l.text for l in pred.text_lines]
    print(idx,pred_text,end='\n')
    print(bboxes)

In [ ]:

# generate_csv(bboxes,pred_text)

In [ ]:
print(lines)

In [ ]:
for idx,line in enumerate(lines):
    print(idx,len(line),[(idx,item.text) for idx,item in enumerate(line)])


In [ ]:
    
import pandas as pd